# スコア集計

各フェーズ（pre, training1-3, post）の正答数・正答率を集計します。


In [ ]:
import json
from collections import defaultdict
from pathlib import Path
import pandas as pd
import numpy as np
from scipy import stats
from scipy.integrate import quad

In [ ]:
# プロジェクトルート
project_root = Path.cwd().parent.parent
base_dir = project_root / "data" / "input"
print(f"データディレクトリ: {base_dir}")

## スコア抽出


In [ ]:
def extract_scores(base_dir: Path) -> dict:
    """
    ログファイルからスコアを抽出する
    """
    data = defaultdict(lambda: defaultdict(dict))

    for group_dir in base_dir.iterdir():
        if not group_dir.is_dir():
            continue
        group = group_dir.name

        for participant_dir in group_dir.iterdir():
            if not participant_dir.is_dir():
                continue
            participant = participant_dir.name

            # Pre系・Test系参加者（テスト用）はスキップ
            if participant.startswith("Pre") or participant.startswith("Test"):
                continue

            for phase_dir in participant_dir.iterdir():
                if not phase_dir.is_dir():
                    continue
                phase = phase_dir.name

                logs_dir = phase_dir / "logs"
                if not logs_dir.exists():
                    continue

                for log_file in logs_dir.glob("*.jsonl"):
                    correct_total = 0
                    question_total = 0

                    with open(log_file, "r") as f:
                        for line in f:
                            line = line.strip()
                            if not line:
                                continue
                            try:
                                event = json.loads(line)
                                if event.get("event") == "answer_submit":
                                    correct_total += event.get("correct_count", 0)
                                    question_total += event.get("total_count", 0)
                                elif event.get("event") == "analog_answer_submit":
                                    correct_total += event.get("correct_count", 0)
                                    question_total += event.get("total_count", 0)
                            except json.JSONDecodeError:
                                pass

                    if question_total > 0:
                        data[group][participant][phase] = {
                            "correct": correct_total,
                            "total": question_total,
                            "rate": correct_total / question_total * 100,
                        }

    return data


data = extract_scores(base_dir)
print(f"グループ数: {len(data)}")
for g in sorted(data.keys()):
    print(f"  グループ {g}: {len(data[g])}名")

## 参加者別スコア一覧


In [ ]:
def create_score_dataframe(data: dict) -> pd.DataFrame:
    """
    スコアデータをDataFrameに変換
    """
    rows = []
    phases = ["pre", "training1", "training2", "training3", "post"]

    for group in sorted(data.keys()):
        for participant in sorted(data[group].keys()):
            p_data = data[group][participant]

            row = {
                "グループ": group,
                "参加者": participant,
            }

            for phase in phases:
                d = p_data.get(phase, {})
                if d:
                    row[f"{phase}_correct"] = d["correct"]
                    row[f"{phase}_total"] = d["total"]
                    row[f"{phase}_rate"] = d["rate"]
                else:
                    row[f"{phase}_correct"] = None
                    row[f"{phase}_total"] = None
                    row[f"{phase}_rate"] = None

            # 伸び計算
            pre = p_data.get("pre", {})
            post = p_data.get("post", {})
            if pre and post:
                row["伸び"] = post["rate"] - pre["rate"]
            else:
                row["伸び"] = None

            rows.append(row)

    return pd.DataFrame(rows)


df = create_score_dataframe(data)
df

In [ ]:
def format_score_table(data: dict) -> pd.DataFrame:
    """
    見やすい形式のスコア表を作成
    """
    rows = []

    for group in sorted(data.keys()):
        for participant in sorted(data[group].keys()):
            p_data = data[group][participant]

            def fmt(d):
                if not d:
                    return "-"
                return f"{d['correct']}/{d['total']} ({d['rate']:.1f}%)"

            pre = p_data.get("pre", {})
            post = p_data.get("post", {})

            if pre and post:
                diff = f"{post['rate'] - pre['rate']:+.1f}%"
            else:
                diff = "-"

            rows.append(
                {
                    "グループ": group,
                    "参加者": participant,
                    "Pre": fmt(p_data.get("pre")),
                    "Training1": fmt(p_data.get("training1")),
                    "Training2": fmt(p_data.get("training2")),
                    "Training3": fmt(p_data.get("training3")),
                    "Post": fmt(p_data.get("post")),
                    "伸び": diff,
                }
            )

    return pd.DataFrame(rows)


score_table = format_score_table(data)
score_table

## グループ別集計


In [ ]:
def compute_group_summary(data: dict) -> pd.DataFrame:
    """
    グループ別の集計（全フェーズ完了者のみ）
    """
    rows = []
    phases = ["pre", "training1", "training2", "training3", "post"]

    for group in sorted(data.keys()):
        totals = {phase: {"correct": 0, "total": 0} for phase in phases}
        complete_count = 0

        for participant in data[group]:
            p_data = data[group][participant]

            # 全フェーズ完了者のみ
            if all(phase in p_data for phase in phases):
                complete_count += 1
                for phase in phases:
                    totals[phase]["correct"] += p_data[phase]["correct"]
                    totals[phase]["total"] += p_data[phase]["total"]

        if complete_count > 0:
            row = {"グループ": group, "n": complete_count}

            for phase in phases:
                c = totals[phase]["correct"]
                t = totals[phase]["total"]
                rate = c / t * 100 if t > 0 else 0
                row[phase] = f"{c}/{t} ({rate:.1f}%)"
                row[f"{phase}_rate"] = rate

            row["伸び"] = f"{row['post_rate'] - row['pre_rate']:+.1f}%"
            rows.append(row)

    df = pd.DataFrame(rows)
    display_cols = [
        "グループ",
        "n",
        "pre",
        "training1",
        "training2",
        "training3",
        "post",
        "伸び",
    ]
    return df[display_cols]


group_summary = compute_group_summary(data)
group_summary

## 参加者数サマリー


In [ ]:
def participant_summary(data: dict) -> pd.DataFrame:
    """
    参加者数のサマリー
    """
    phases = ["pre", "training1", "training2", "training3", "post"]
    rows = []

    for group in sorted(data.keys()):
        total = len(data[group])
        complete = sum(
            1 for p in data[group] if all(phase in data[group][p] for phase in phases)
        )
        rows.append(
            {
                "グループ": group,
                "全参加者": total,
                "完了者": complete,
                "未完了": total - complete,
            }
        )

    return pd.DataFrame(rows)


participant_summary(data)

## CSV出力


In [ ]:
# 必要に応じてCSV出力
# output_dir = project_root / "data" / "output"
# output_dir.mkdir(parents=True, exist_ok=True)
# df.to_csv(output_dir / "score_summary.csv", index=False, encoding='utf-8-sig')
# print(f"保存先: {output_dir / 'score_summary.csv'}")

## 統計分析

### 1. 記述統計


In [ ]:
# Pre/Post正答率と伸びをグループ別に計算
def compute_descriptive_stats(data: dict) -> dict:
    """
    記述統計を計算
    """
    stats_data = {}

    for group in sorted(data.keys()):
        pre_rates = []
        post_rates = []
        gains = []

        for participant in data[group]:
            p_data = data[group][participant]
            if "pre" in p_data and "post" in p_data:
                pre = p_data["pre"]["rate"]
                post = p_data["post"]["rate"]
                pre_rates.append(pre)
                post_rates.append(post)
                gains.append(post - pre)

        stats_data[group] = {
            "pre": np.array(pre_rates),
            "post": np.array(post_rates),
            "gain": np.array(gains),
            "n": len(pre_rates),
        }

    return stats_data


stats_data = compute_descriptive_stats(data)

# 記述統計テーブル
desc_rows = []
for group in sorted(stats_data.keys()):
    d = stats_data[group]
    desc_rows.append(
        {
            "グループ": group,
            "n": d["n"],
            "Pre M(SD)": f"{np.mean(d['pre']):.1f}% ({np.std(d['pre'], ddof=1):.1f}%)",
            "Post M(SD)": f"{np.mean(d['post']):.1f}% ({np.std(d['post'], ddof=1):.1f}%)",
            "伸び M(SD)": f"{np.mean(d['gain']):+.1f}% ({np.std(d['gain'], ddof=1):.1f}%)",
            "伸び範囲": f"{np.min(d['gain']):+.1f}% ~ {np.max(d['gain']):+.1f}%",
        }
    )

desc_df = pd.DataFrame(desc_rows)
print("【記述統計】")
desc_df

### 2. 正規性検定（Shapiro-Wilk検定）

パラメトリック検定（t検定）を使用できるか確認します。

- p > 0.05 → 正規性を仮定できる → t検定を使用
- p ≤ 0.05 → 正規性を仮定できない → ノンパラメトリック検定を使用


In [ ]:
# Shapiro-Wilk検定
print("【正規性検定 (Shapiro-Wilk)】")
print("=" * 60)

normality_results = []
for group in sorted(stats_data.keys()):
    d = stats_data[group]

    # Pre
    stat_pre, p_pre = stats.shapiro(d["pre"])
    # Post
    stat_post, p_post = stats.shapiro(d["post"])
    # 伸び
    stat_gain, p_gain = stats.shapiro(d["gain"])

    normality_results.append(
        {
            "グループ": group,
            "Pre W": f"{stat_pre:.3f}",
            "Pre p": f"{p_pre:.3f}",
            "Pre 正規性": "○" if p_pre > 0.05 else "×",
            "Post W": f"{stat_post:.3f}",
            "Post p": f"{p_post:.3f}",
            "Post 正規性": "○" if p_post > 0.05 else "×",
            "伸び W": f"{stat_gain:.3f}",
            "伸び p": f"{p_gain:.3f}",
            "伸び 正規性": "○" if p_gain > 0.05 else "×",
        }
    )

norm_df = pd.DataFrame(normality_results)
norm_df

### 3. 各群内の学習効果検証

各群でPre→Postで有意な変化があったかを検証します。

- **パラメトリック**: 対応のあるt検定 (paired t-test)
- **ノンパラメトリック**: Wilcoxon符号順位検定


In [ ]:
# 各群内の学習効果検証
print("【各群内の学習効果 (Pre vs Post)】")
print("=" * 60)

within_results = []
for group in sorted(stats_data.keys()):
    d = stats_data[group]
    pre = d["pre"]
    post = d["post"]

    # 対応のあるt検定（片側: Post > Pre）
    t_stat, t_p = stats.ttest_rel(post, pre, alternative="greater")

    # Wilcoxon符号順位検定（片側: Post > Pre）
    w_stat, w_p = stats.wilcoxon(post - pre, alternative="greater")

    # 効果量 (Cohen's d for paired samples)
    diff = post - pre
    cohens_d = np.mean(diff) / np.std(diff, ddof=1)

    within_results.append(
        {
            "グループ": group,
            "n": d["n"],
            "平均差": f"{np.mean(diff):+.1f}%",
            "t値": f"{t_stat:.3f}",
            "t検定 p": f"{t_p:.3f}",
            "Wilcoxon p": f"{w_p:.3f}",
            "Cohen's d": f"{cohens_d:.3f}",
            "効果量": "大"
            if abs(cohens_d) >= 0.8
            else "中"
            if abs(cohens_d) >= 0.5
            else "小"
            if abs(cohens_d) >= 0.2
            else "なし",
        }
    )

    print(f"\n■ グループ {group}")
    print(
        f"  Pre → Post: {np.mean(pre):.1f}% → {np.mean(post):.1f}% (差: {np.mean(diff):+.1f}%)"
    )
    print(f"  対応のあるt検定: t = {t_stat:.3f}, p = {t_p:.3f}")
    print(f"  Wilcoxon符号順位検定: p = {w_p:.3f}")
    print(f"  効果量 (Cohen's d): {cohens_d:.3f}")

within_df = pd.DataFrame(within_results)
print("\n")
within_df

### 4. 群間比較（伸びの差）

A群とB群で「伸び」に有意差があるかを検証します。

- **パラメトリック**: 独立t検定 (independent t-test)
- **ノンパラメトリック**: Mann-Whitney U検定


In [ ]:
# 群間比較（伸びの差）
print("【群間比較 (A群 vs B群の伸び)】")
print("=" * 60)

gain_A = stats_data["A"]["gain"]
gain_B = stats_data["B"]["gain"]

# 独立t検定（片側: B群 > A群）
t_stat, t_p = stats.ttest_ind(gain_A, gain_B, alternative="less")

# Mann-Whitney U検定（片側: B群 > A群）
u_stat, u_p = stats.mannwhitneyu(gain_A, gain_B, alternative="less")

# 効果量 (Cohen's d for independent samples)
pooled_std = np.sqrt(
    (
        (len(gain_A) - 1) * np.var(gain_A, ddof=1)
        + (len(gain_B) - 1) * np.var(gain_B, ddof=1)
    )
    / (len(gain_A) + len(gain_B) - 2)
)
cohens_d = (np.mean(gain_A) - np.mean(gain_B)) / pooled_std

print(f"A群の伸び: M = {np.mean(gain_A):+.1f}%, SD = {np.std(gain_A, ddof=1):.1f}%")
print(f"B群の伸び: M = {np.mean(gain_B):+.1f}%, SD = {np.std(gain_B, ddof=1):.1f}%")
print(f"群間差: {np.mean(gain_A) - np.mean(gain_B):+.1f}%")
print()
print(f"独立t検定: t = {t_stat:.3f}, p = {t_p:.3f}")
print(f"Mann-Whitney U検定: U = {u_stat:.1f}, p = {u_p:.3f}")
print(f"効果量 (Cohen's d): {cohens_d:.3f}")

effect_size = (
    "大"
    if abs(cohens_d) >= 0.8
    else "中"
    if abs(cohens_d) >= 0.5
    else "小"
    if abs(cohens_d) >= 0.2
    else "なし"
)
print(f"効果量の解釈: {effect_size}")

# 結果のサマリー
between_df = pd.DataFrame(
    [
        {
            "比較": "A群 vs B群 (伸び)",
            "A群 M(SD)": f"{np.mean(gain_A):+.1f}% ({np.std(gain_A, ddof=1):.1f}%)",
            "B群 M(SD)": f"{np.mean(gain_B):+.1f}% ({np.std(gain_B, ddof=1):.1f}%)",
            "群間差": f"{np.mean(gain_A) - np.mean(gain_B):+.1f}%",
            "t値": f"{t_stat:.3f}",
            "t検定 p": f"{t_p:.3f}",
            "U値": f"{u_stat:.1f}",
            "Mann-Whitney p": f"{u_p:.3f}",
            "Cohen's d": f"{cohens_d:.3f}",
            "効果量": effect_size,
        }
    ]
)
print("\n")
between_df

### 5. 結果のまとめ


In [ ]:
# 結果のまとめ
print("=" * 70)
print("【統計分析結果のまとめ】")
print("=" * 70)

print("\n■ 記述統計")
print(
    f"  A群 (n=10): Pre {np.mean(stats_data['A']['pre']):.1f}% → Post {np.mean(stats_data['A']['post']):.1f}% (伸び: {np.mean(stats_data['A']['gain']):+.1f}%)"
)
print(
    f"  B群 (n=10): Pre {np.mean(stats_data['B']['pre']):.1f}% → Post {np.mean(stats_data['B']['post']):.1f}% (伸び: {np.mean(stats_data['B']['gain']):+.1f}%)"
)

print("\n■ 各群内の学習効果")
for group in ["A", "B"]:
    d = stats_data[group]
    t_stat, t_p = stats.ttest_rel(d["post"], d["pre"], alternative="greater")
    sig = "有意" if t_p < 0.05 else "有意でない"
    print(f"  {group}群: t = {t_stat:.3f}, p = {t_p:.3f} → {sig}")

print("\n■ 群間比較（伸びの差）")
t_stat, t_p = stats.ttest_ind(gain_A, gain_B, alternative="less")
sig = "有意差あり" if t_p < 0.05 else "有意差なし"
print(f"  独立t検定: t = {t_stat:.3f}, p = {t_p:.3f} → {sig}")

print("\n■ 結論")
if t_p < 0.05:
    if np.mean(gain_B) > np.mean(gain_A):
        print("  B群の学習法がA群より有意に効果的であった")
    else:
        print("  A群の学習法がB群より有意に効果的であった")
else:
    print("  A群とB群の学習法に統計的有意差は認められなかった")
    print(
        f"  (ただし、B群の方が伸びが大きい傾向: +{np.mean(gain_B) - np.mean(gain_A):.1f}%)"
    )

---

## 追加の統計分析

### 6. 混合分散分析（Mixed ANOVA）

グループ（被験者間要因）× 時点（被験者内要因）の二元配置分散分析。
交互作用が有意であれば、「伸び方がグループ間で異なる」ことを示す。


In [ ]:
# Mixed ANOVA用のデータ準備
# Long format に変換
anova_rows = []
for group in ["A", "B"]:
    for i, (pre, post) in enumerate(
        zip(stats_data[group]["pre"], stats_data[group]["post"])
    ):
        anova_rows.append(
            {"subject": f"{group}_{i}", "group": group, "time": "pre", "score": pre}
        )
        anova_rows.append(
            {"subject": f"{group}_{i}", "group": group, "time": "post", "score": post}
        )

anova_df = pd.DataFrame(anova_rows)
print("【Mixed ANOVA用データ（Long format）】")
print(anova_df.head(10))

In [ ]:
# Mixed ANOVA (手動計算)
# pingouin がインストールされていない場合の代替実装

print("【混合分散分析 (Mixed ANOVA)】")
print("=" * 70)

# データ準備
pre_A = stats_data["A"]["pre"]
post_A = stats_data["A"]["post"]
pre_B = stats_data["B"]["pre"]
post_B = stats_data["B"]["post"]

n_A, n_B = len(pre_A), len(pre_B)
N = n_A + n_B  # 総人数

# 全体平均
grand_mean = np.mean(np.concatenate([pre_A, post_A, pre_B, post_B]))

# グループ平均
mean_A = np.mean(np.concatenate([pre_A, post_A]))
mean_B = np.mean(np.concatenate([pre_B, post_B]))

# 時点平均
mean_pre = np.mean(np.concatenate([pre_A, pre_B]))
mean_post = np.mean(np.concatenate([post_A, post_B]))

# セル平均
mean_A_pre = np.mean(pre_A)
mean_A_post = np.mean(post_A)
mean_B_pre = np.mean(pre_B)
mean_B_post = np.mean(post_B)

# 平方和の計算
# グループ主効果 (Between-subjects)
SS_group = 2 * (n_A * (mean_A - grand_mean) ** 2 + n_B * (mean_B - grand_mean) ** 2)

# 時点主効果 (Within-subjects)
SS_time = N * ((mean_pre - grand_mean) ** 2 + (mean_post - grand_mean) ** 2)

# 交互作用
SS_interaction = n_A * (
    (mean_A_pre - mean_A - mean_pre + grand_mean) ** 2
    + (mean_A_post - mean_A - mean_post + grand_mean) ** 2
) + n_B * (
    (mean_B_pre - mean_B - mean_pre + grand_mean) ** 2
    + (mean_B_post - mean_B - mean_post + grand_mean) ** 2
)

# 被験者間誤差
subject_means_A = (pre_A + post_A) / 2
subject_means_B = (pre_B + post_B) / 2
SS_subjects = 2 * (
    np.sum((subject_means_A - mean_A) ** 2) + np.sum((subject_means_B - mean_B) ** 2)
)

# 被験者内誤差 (残差)
SS_error_within = 0
for i in range(n_A):
    SS_error_within += (pre_A[i] - subject_means_A[i] - mean_pre + mean_A) ** 2
    SS_error_within += (post_A[i] - subject_means_A[i] - mean_post + mean_A) ** 2
for i in range(n_B):
    SS_error_within += (pre_B[i] - subject_means_B[i] - mean_pre + mean_B) ** 2
    SS_error_within += (post_B[i] - subject_means_B[i] - mean_post + mean_B) ** 2

# 自由度
df_group = 1
df_time = 1
df_interaction = 1
df_subjects = N - 2  # 被験者間誤差
df_error_within = N - 2  # 被験者内誤差

# 平均平方
MS_group = SS_group / df_group
MS_time = SS_time / df_time
MS_interaction = SS_interaction / df_interaction
MS_subjects = SS_subjects / df_subjects
MS_error_within = SS_error_within / df_error_within

# F値
F_group = MS_group / MS_subjects
F_time = MS_time / MS_error_within
F_interaction = MS_interaction / MS_error_within

# p値
p_group = 1 - stats.f.cdf(F_group, df_group, df_subjects)
p_time = 1 - stats.f.cdf(F_time, df_time, df_error_within)
p_interaction = 1 - stats.f.cdf(F_interaction, df_interaction, df_error_within)

# 効果量 (partial η²)
eta2_group = SS_group / (SS_group + SS_subjects)
eta2_time = SS_time / (SS_time + SS_error_within)
eta2_interaction = SS_interaction / (SS_interaction + SS_error_within)

print("\n■ グループ主効果 (A vs B)")
print(
    f"  F({df_group}, {df_subjects}) = {F_group:.3f}, p = {p_group:.3f}, η²p = {eta2_group:.3f}"
)
print(f"  → {'有意' if p_group < 0.05 else '有意でない'}")

print("\n■ 時点主効果 (Pre vs Post)")
print(
    f"  F({df_time}, {df_error_within}) = {F_time:.3f}, p = {p_time:.3f}, η²p = {eta2_time:.3f}"
)
print(f"  → {'有意' if p_time < 0.05 else '有意でない'}")

print("\n■ 交互作用 (グループ × 時点)")
print(
    f"  F({df_interaction}, {df_error_within}) = {F_interaction:.3f}, p = {p_interaction:.3f}, η²p = {eta2_interaction:.3f}"
)
print(f"  → {'有意' if p_interaction < 0.05 else '有意でない'}")

# 結果テーブル
anova_results = pd.DataFrame(
    [
        {
            "効果": "グループ (A vs B)",
            "SS": f"{SS_group:.2f}",
            "df": df_group,
            "MS": f"{MS_group:.2f}",
            "F": f"{F_group:.3f}",
            "p": f"{p_group:.3f}",
            "η²p": f"{eta2_group:.3f}",
            "判定": "有意" if p_group < 0.05 else "n.s.",
        },
        {
            "効果": "時点 (Pre vs Post)",
            "SS": f"{SS_time:.2f}",
            "df": df_time,
            "MS": f"{MS_time:.2f}",
            "F": f"{F_time:.3f}",
            "p": f"{p_time:.3f}",
            "η²p": f"{eta2_time:.3f}",
            "判定": "有意" if p_time < 0.05 else "n.s.",
        },
        {
            "効果": "交互作用",
            "SS": f"{SS_interaction:.2f}",
            "df": df_interaction,
            "MS": f"{MS_interaction:.2f}",
            "F": f"{F_interaction:.3f}",
            "p": f"{p_interaction:.3f}",
            "η²p": f"{eta2_interaction:.3f}",
            "判定": "有意" if p_interaction < 0.05 else "n.s.",
        },
    ]
)
print("\n")
anova_results

### 7. 共分散分析（ANCOVA）

Preスコアを共変量として統制し、Postスコアをグループ間で比較。
→ 事前の個人差を統制した上で、学習効果の群間差を検証できる。


In [ ]:
# ANCOVA (共分散分析)
# Preスコアを共変量、Postスコアを従属変数、グループを独立変数

print("【共分散分析 (ANCOVA)】")
print("=" * 70)
print("従属変数: Postスコア")
print("独立変数: グループ (A vs B)")
print("共変量: Preスコア")
print()

# データ準備
pre_all = np.concatenate([pre_A, pre_B])
post_all = np.concatenate([post_A, post_B])
group_all = np.array([0] * n_A + [1] * n_B)  # A=0, B=1

# 回帰分析によるANCOVA
# Model: Post = β0 + β1*Pre + β2*Group + ε

# デザイン行列
X = np.column_stack([np.ones(N), pre_all, group_all])
y = post_all

# 最小二乗法
beta = np.linalg.lstsq(X, y, rcond=None)[0]
y_pred = X @ beta
residuals = y - y_pred
SS_residual = np.sum(residuals**2)
df_residual = N - 3  # N - パラメータ数

# グループ効果の検定 (β2)
# フルモデル vs 縮小モデル（グループなし）
X_reduced = np.column_stack([np.ones(N), pre_all])
beta_reduced = np.linalg.lstsq(X_reduced, y, rcond=None)[0]
y_pred_reduced = X_reduced @ beta_reduced
SS_residual_reduced = np.sum((y - y_pred_reduced) ** 2)

SS_group_effect = SS_residual_reduced - SS_residual
df_group_effect = 1
MS_group_effect = SS_group_effect / df_group_effect
MS_residual = SS_residual / df_residual

F_ancova = MS_group_effect / MS_residual
p_ancova = 1 - stats.f.cdf(F_ancova, df_group_effect, df_residual)

# 効果量 (partial η²)
eta2_ancova = SS_group_effect / (SS_group_effect + SS_residual)

# 調整済み平均
mean_pre_all = np.mean(pre_all)
adj_mean_A = beta[0] + beta[1] * mean_pre_all + beta[2] * 0
adj_mean_B = beta[0] + beta[1] * mean_pre_all + beta[2] * 1

print("■ 回帰係数")
print(f"  切片 (β0): {beta[0]:.3f}")
print(f"  Pre効果 (β1): {beta[1]:.3f}")
print(f"  グループ効果 (β2): {beta[2]:.3f}")

print("\n■ 調整済み平均 (Preを統制)")
print(f"  A群: {adj_mean_A:.1f}%")
print(f"  B群: {adj_mean_B:.1f}%")
print(f"  差: {adj_mean_B - adj_mean_A:+.1f}%")

print("\n■ グループ効果の検定")
print(f"  F({df_group_effect}, {df_residual}) = {F_ancova:.3f}, p = {p_ancova:.3f}")
print(f"  η²p = {eta2_ancova:.3f}")
print(f"  → {'有意' if p_ancova < 0.05 else '有意でない'}")

# 結果テーブル
ancova_results = pd.DataFrame(
    [
        {
            "効果": "グループ (Preを統制)",
            "調整済み平均A": f"{adj_mean_A:.1f}%",
            "調整済み平均B": f"{adj_mean_B:.1f}%",
            "調整済み差": f"{adj_mean_B - adj_mean_A:+.1f}%",
            "F": f"{F_ancova:.3f}",
            "p": f"{p_ancova:.3f}",
            "η²p": f"{eta2_ancova:.3f}",
            "判定": "有意" if p_ancova < 0.05 else "n.s.",
        }
    ]
)
print("\n")
ancova_results

### 8. 効果量の信頼区間（ブートストラップ法）

Cohen's dの95%信頼区間を算出し、効果の不確実性を評価する。


In [ ]:
# ブートストラップによる効果量の信頼区間
print("【効果量の信頼区間 (ブートストラップ法)】")
print("=" * 70)

np.random.seed(42)
n_bootstrap = 10000


def compute_cohens_d(x, y):
    """独立サンプルのCohen's d"""
    nx, ny = len(x), len(y)
    pooled_std = np.sqrt(
        ((nx - 1) * np.var(x, ddof=1) + (ny - 1) * np.var(y, ddof=1)) / (nx + ny - 2)
    )
    return (np.mean(x) - np.mean(y)) / pooled_std if pooled_std > 0 else 0


# 元のCohen's d
original_d = compute_cohens_d(gain_A, gain_B)

# ブートストラップ
bootstrap_ds = []
for _ in range(n_bootstrap):
    # リサンプリング
    idx_A = np.random.choice(len(gain_A), size=len(gain_A), replace=True)
    idx_B = np.random.choice(len(gain_B), size=len(gain_B), replace=True)
    boot_A = gain_A[idx_A]
    boot_B = gain_B[idx_B]
    bootstrap_ds.append(compute_cohens_d(boot_A, boot_B))

bootstrap_ds = np.array(bootstrap_ds)

# 95%信頼区間 (パーセンタイル法)
ci_lower = np.percentile(bootstrap_ds, 2.5)
ci_upper = np.percentile(bootstrap_ds, 97.5)

print("■ 群間比較 (A群 vs B群の伸び)")
print(f"  Cohen's d = {original_d:.3f}")
print(f"  95% CI = [{ci_lower:.3f}, {ci_upper:.3f}]")
print()
print(f"  解釈: 信頼区間が0を{'含む' if ci_lower <= 0 <= ci_upper else '含まない'}")
if ci_lower <= 0 <= ci_upper:
    print("        → 効果がないという可能性を排除できない")
else:
    print("        → 効果があると解釈できる")

# 各群内のブートストラップ
print("\n■ 各群内の学習効果")
for group_name, pre, post in [("A", pre_A, post_A), ("B", pre_B, post_B)]:
    diff = post - pre
    original_d_within = np.mean(diff) / np.std(diff, ddof=1)

    bootstrap_ds_within = []
    for _ in range(n_bootstrap):
        idx = np.random.choice(len(diff), size=len(diff), replace=True)
        boot_diff = diff[idx]
        if np.std(boot_diff, ddof=1) > 0:
            bootstrap_ds_within.append(np.mean(boot_diff) / np.std(boot_diff, ddof=1))

    bootstrap_ds_within = np.array(bootstrap_ds_within)
    ci_lower_within = np.percentile(bootstrap_ds_within, 2.5)
    ci_upper_within = np.percentile(bootstrap_ds_within, 97.5)

    print(
        f"  {group_name}群: d = {original_d_within:.3f}, 95% CI = [{ci_lower_within:.3f}, {ci_upper_within:.3f}]"
    )

### 9. ベイズ統計（Bayes Factor）

「差がある」vs「差がない」の証拠の強さを評価。

- BF10 > 3: 差がある証拠が中程度
- BF10 > 10: 差がある証拠が強い
- BF10 < 1/3: 差がない証拠が中程度
- BF10 < 1/10: 差がない証拠が強い


In [ ]:
# ベイズファクター (JZS prior を使用した近似計算)
# Rouder et al. (2009) の方法に基づく簡易実装

print("【ベイズファクター (Bayes Factor)】")
print("=" * 70)


def bayesian_t_test(x, y, r=0.707):
    """
    独立サンプルのベイズt検定
    JZS prior (Cauchy prior with scale r) を使用

    Rouder et al. (2009) の近似式を使用
    """
    nx, ny = len(x), len(y)
    n = nx + ny

    # t統計量
    pooled_var = ((nx - 1) * np.var(x, ddof=1) + (ny - 1) * np.var(y, ddof=1)) / (n - 2)
    se = np.sqrt(pooled_var * (1 / nx + 1 / ny))
    t = (np.mean(x) - np.mean(y)) / se

    df = n - 2

    # BF10 の近似計算 (Wagenmakers et al., 2007 の簡易版)
    # BF01 ≈ (1 + t²/df)^(-(df+1)/2) * sqrt(df/(df+1)) * (1 + neff*r²)^(1/2)
    # ただし neff = nx*ny/(nx+ny)

    neff = (nx * ny) / (nx + ny)

    # より正確な近似: Savage-Dickey density ratio
    # 帰無仮説下の t の密度
    from scipy.stats import t as t_dist

    # BF01 の計算 (Wetzels et al., 2011 の方法)
    # 数値積分による計算
    def integrand(delta, t_val, n_eff, df_val, r_scale):
        # 尤度
        likelihood = t_dist.pdf(t_val, df_val, loc=delta * np.sqrt(n_eff))
        # 事前分布 (Cauchy)
        prior = stats.cauchy.pdf(delta, scale=r_scale)
        return likelihood * prior

    from scipy.integrate import quad

    # 帰無仮説下の尤度
    likelihood_h0 = t_dist.pdf(t, df)

    # 対立仮説下の尤度（周辺尤度）
    marginal_likelihood_h1, _ = quad(
        lambda delta: integrand(delta, t, neff, df, r), -np.inf, np.inf
    )

    # Bayes Factor
    bf10 = marginal_likelihood_h1 / likelihood_h0
    bf01 = 1 / bf10

    return t, df, bf10, bf01


# 群間比較
t_val, df_val, bf10, bf01 = bayesian_t_test(gain_A, gain_B)

print("■ 群間比較 (A群 vs B群の伸び)")
print(f"  t({df_val}) = {t_val:.3f}")
print(f"  BF10 = {bf10:.3f} (H1: 差がある vs H0: 差がない)")
print(f"  BF01 = {bf01:.3f} (H0: 差がない vs H1: 差がある)")
print()

# 解釈
if bf10 > 10:
    interpretation = "差がある強い証拠"
elif bf10 > 3:
    interpretation = "差がある中程度の証拠"
elif bf10 > 1:
    interpretation = "差がある弱い証拠"
elif bf01 > 10:
    interpretation = "差がない強い証拠"
elif bf01 > 3:
    interpretation = "差がない中程度の証拠"
else:
    interpretation = "どちらとも言えない (inconclusive)"

print(f"  解釈: {interpretation}")

# 各群内のベイズt検定
print("\n■ 各群内の学習効果 (Pre vs Post)")
for group_name, pre, post in [("A", pre_A, post_A), ("B", pre_B, post_B)]:
    diff = post - pre
    n = len(diff)
    t_within = np.mean(diff) / (np.std(diff, ddof=1) / np.sqrt(n))
    df_within = n - 1

    # 1サンプルt検定のBF
    # 簡易計算
    from scipy.stats import t as t_dist

    likelihood_h0_within = t_dist.pdf(t_within, df_within)

    def integrand_1sample(delta, t_val, n_val, df_val, r_scale=0.707):
        likelihood = t_dist.pdf(t_val, df_val, loc=delta * np.sqrt(n_val))
        prior = stats.cauchy.pdf(delta, scale=r_scale)
        return likelihood * prior

    marginal_h1_within, _ = quad(
        lambda delta: integrand_1sample(delta, t_within, n, df_within), -np.inf, np.inf
    )

    bf10_within = marginal_h1_within / likelihood_h0_within
    bf01_within = 1 / bf10_within

    print(
        f"  {group_name}群: t({df_within}) = {t_within:.3f}, BF10 = {bf10_within:.3f}, BF01 = {bf01_within:.3f}"
    )

### 10. 天井効果・床効果の検討

高得点者（Pre 90%以上）は伸びにくい可能性がある。この影響を分析する。


In [ ]:
# 天井効果・床効果の検討
print("【天井効果・床効果の検討】")
print("=" * 70)

# Pre得点と伸びの相関
all_pre = np.concatenate([pre_A, pre_B])
all_gain = np.concatenate([gain_A, gain_B])

corr, p_corr = stats.pearsonr(all_pre, all_gain)
print("■ Pre得点と伸びの相関")
print(f"  r = {corr:.3f}, p = {p_corr:.3f}")
if corr < 0 and p_corr < 0.05:
    print("  → 有意な負の相関: Pre得点が高い人ほど伸びにくい（天井効果の可能性）")
elif corr < 0:
    print("  → 負の相関傾向: Pre得点が高い人ほど伸びにくい傾向（天井効果の可能性）")

# 高得点者・低得点者の分析
threshold_high = 85  # 高得点の閾値
threshold_low = 65  # 低得点の閾値

print(f"\n■ Pre得点による層別分析 (閾値: 高≥{threshold_high}%, 低≤{threshold_low}%)")

for group_name, pre, gain in [("A", pre_A, gain_A), ("B", pre_B, gain_B)]:
    high_mask = pre >= threshold_high
    low_mask = pre <= threshold_low
    mid_mask = ~high_mask & ~low_mask

    print(f"\n  {group_name}群:")
    print(
        f"    高得点者 (≥{threshold_high}%): n={np.sum(high_mask)}, 伸び平均 = {np.mean(gain[high_mask]):+.1f}%"
        if np.sum(high_mask) > 0
        else "    高得点者: n=0"
    )
    print(
        f"    中得点者: n={np.sum(mid_mask)}, 伸び平均 = {np.mean(gain[mid_mask]):+.1f}%"
        if np.sum(mid_mask) > 0
        else "    中得点者: n=0"
    )
    print(
        f"    低得点者 (≤{threshold_low}%): n={np.sum(low_mask)}, 伸び平均 = {np.mean(gain[low_mask]):+.1f}%"
        if np.sum(low_mask) > 0
        else "    低得点者: n=0"
    )

# 天井効果を除外した分析
print(f"\n■ 高得点者 (Pre≥{threshold_high}%) を除外した分析")
mask_A_no_ceiling = pre_A < threshold_high
mask_B_no_ceiling = pre_B < threshold_high

gain_A_no_ceiling = gain_A[mask_A_no_ceiling]
gain_B_no_ceiling = gain_B[mask_B_no_ceiling]

if len(gain_A_no_ceiling) > 1 and len(gain_B_no_ceiling) > 1:
    print(
        f"  A群: n={len(gain_A_no_ceiling)}, 伸び平均 = {np.mean(gain_A_no_ceiling):+.1f}%"
    )
    print(
        f"  B群: n={len(gain_B_no_ceiling)}, 伸び平均 = {np.mean(gain_B_no_ceiling):+.1f}%"
    )

    t_no_ceiling, p_no_ceiling = stats.ttest_ind(gain_A_no_ceiling, gain_B_no_ceiling, alternative="less")
    print(f"  独立t検定: t = {t_no_ceiling:.3f}, p = {p_no_ceiling:.3f}")
else:
    print("  除外後のサンプルサイズが小さすぎます")

### 11. 統計分析の総合まとめ


In [ ]:
# 統計分析の総合まとめ
print("=" * 70)
print("【統計分析の総合まとめ】")
print("=" * 70)

print("""
┌─────────────────────────────────────────────────────────────────────┐
│  分析手法              │  結果                    │  結論          │
├─────────────────────────────────────────────────────────────────────┤
│  1. 記述統計           │  A群: +1.0%, B群: +3.5%  │  B群が大きい   │
│  2. 対応のあるt検定    │  両群ともp > 0.05        │  有意でない    │
│  3. 独立t検定          │  p = 0.667               │  有意差なし    │
│  4. Mann-Whitney U     │  p = 0.536               │  有意差なし    │
│  5. 混合ANOVA          │  交互作用 p > 0.05       │  有意でない    │
│  6. ANCOVA             │  p > 0.05                │  有意差なし    │
│  7. ブートストラップ   │  95%CI が 0 を含む       │  効果不確実    │
│  8. ベイズファクター   │  BF01 > 1                │  差なしを支持  │
│  9. 天井効果           │  高得点者は伸びにくい    │  分析に注意    │
└─────────────────────────────────────────────────────────────────────┘
""")

print("■ 総合結論")
print("  ・A群とB群の学習法に統計的有意差は認められなかった")
print("  ・ベイズ分析では「差がない」ことを支持する証拠がある")
print("  ・B群の方が伸びが大きい傾向 (+2.5%) があるが、有意ではない")
print()
print("■ 注意点")
print("  ・サンプルサイズが小さい (各群n=10) ため、検出力が低い可能性")
print("  ・天井効果により高得点者の伸びが制限されている可能性")
print("  ・より大きなサンプルサイズでの追試が望ましい")

---

## 有意差が見られなかった原因の分析

### 12. 検出力分析（Power Analysis）

現在のサンプルサイズ（各群n=10）で、どの程度の効果を検出できるかを分析する。


In [ ]:
from scipy.stats import nct  # 非心t分布

# 検出力分析 (Power Analysis)
print("【検出力分析 (Power Analysis)】")
print("=" * 70)


def power_t_test_ind(n1, n2, d, alpha=0.05):
    """
    独立t検定の検出力を計算

    Parameters:
    -----------
    n1, n2 : int - 各群のサンプルサイズ
    d : float - 効果量 (Cohen's d)
    alpha : float - 有意水準

    Returns:
    --------
    power : float - 検出力 (1 - β)
    """
    df = n1 + n2 - 2
    # 非心度パラメータ
    ncp = d * np.sqrt((n1 * n2) / (n1 + n2))
    # 臨界値
    t_crit = stats.t.ppf(1 - alpha / 2, df)
    # 検出力 = 1 - β
    power = 1 - nct.cdf(t_crit, df, ncp) + nct.cdf(-t_crit, df, ncp)
    return power


def required_sample_size(d, power=0.80, alpha=0.05):
    """
    目標検出力を達成するために必要なサンプルサイズを計算
    """
    for n in range(5, 1000):
        if power_t_test_ind(n, n, d, alpha) >= power:
            return n
    return ">1000"


# 現在のデータ
n_current = 10
d_observed = abs(compute_cohens_d(gain_A, gain_B))  # 0.196

print("■ 現在の状況")
print(f"  サンプルサイズ: 各群 n = {n_current}")
print(f"  観測された効果量: Cohen's d = {d_observed:.3f}")
print("  有意水準: α = 0.05")

# 現在の検出力
power_current = power_t_test_ind(n_current, n_current, d_observed)
print("\n■ 現在の検出力")
print(f"  検出力 (1-β) = {power_current:.3f} ({power_current * 100:.1f}%)")
print(
    f"  → 観測された効果 (d={d_observed:.3f}) を検出できる確率は {power_current * 100:.1f}%"
)

# 様々な効果量での検出力
print("\n■ 効果量別の検出力 (n=10)")
effect_sizes = [0.2, 0.5, 0.8, 1.0, 1.2]
for d in effect_sizes:
    p = power_t_test_ind(n_current, n_current, d)
    label = "小" if d == 0.2 else "中" if d == 0.5 else "大" if d == 0.8 else ""
    print(f"  d = {d:.1f} ({label}): 検出力 = {p:.3f} ({p * 100:.1f}%)")

# 80%検出力に必要なサンプルサイズ
print("\n■ 80%検出力を達成するために必要なサンプルサイズ")
for d in [0.2, 0.5, 0.8, d_observed]:
    n_req = required_sample_size(d)
    label = "小" if d == 0.2 else "中" if d == 0.5 else "大" if d == 0.8 else "観測値"
    print(f"  d = {d:.3f} ({label}): 各群 n = {n_req}")

# 検出力曲線のデータ
print("\n■ サンプルサイズと検出力の関係 (d=0.5, 中程度の効果)")
sample_sizes = [10, 20, 30, 50, 100]
print("  n\t検出力")
for n in sample_sizes:
    p = power_t_test_ind(n, n, 0.5)
    print(f"  {n}\t{p:.3f} ({p * 100:.1f}%)")

# 結論
print("\n" + "=" * 70)
print("【検出力分析の結論】")
print("=" * 70)
print(f"""
1. 現在の検出力は非常に低い ({power_current * 100:.1f}%)
   → 実際に効果があっても検出できない可能性が高い

2. 観測された効果量 (d={d_observed:.3f}) を80%の確率で検出するには
   各群 n={required_sample_size(d_observed)} が必要

3. 中程度の効果 (d=0.5) を検出するには各群 n={required_sample_size(0.5)} が必要

4. 現在のn=10では、大きな効果 (d≥0.8) しか検出できない
   → 検出力 = {power_t_test_ind(10, 10, 0.8) * 100:.1f}%
""")

### 13. 個人差・ばらつきの詳細分析

個人差の大きさと分布を詳しく分析し、有意差が見られなかった原因を探る。


In [ ]:
# 個人差・ばらつきの詳細分析
print("【個人差・ばらつきの詳細分析】")
print("=" * 70)

# 1. 基本的なばらつき指標
print("\n■ 伸びのばらつき指標")
for group_name, gain in [('A', gain_A), ('B', gain_B)]:
    print(f"\n  {group_name}群:")
    print(f"    平均 (M): {np.mean(gain):+.1f}%")
    print(f"    標準偏差 (SD): {np.std(gain, ddof=1):.1f}%")
    print(f"    範囲: {np.min(gain):+.1f}% ~ {np.max(gain):+.1f}%")
    print(f"    四分位範囲 (IQR): {np.percentile(gain, 25):.1f}% ~ {np.percentile(gain, 75):.1f}%")
    print(f"    変動係数 (CV): {np.std(gain, ddof=1) / abs(np.mean(gain)) * 100:.1f}%" if np.mean(gain) != 0 else "    変動係数 (CV): N/A")

# 2. 伸びの分布分析
print("\n\n■ 伸びの分布")
print("  " + "-" * 50)
bins = [(-30, -20), (-20, -10), (-10, 0), (0, 10), (10, 20), (20, 30)]
print(f"  {'範囲':<15} {'A群':>8} {'B群':>8}")
print("  " + "-" * 50)
for low, high in bins:
    count_A = np.sum((gain_A > low) & (gain_A <= high))
    count_B = np.sum((gain_B > low) & (gain_B <= high))
    label = f"{low:+d}% ~ {high:+d}%"
    bar_A = "█" * count_A
    bar_B = "█" * count_B
    print(f"  {label:<15} {count_A:>3} {bar_A:<5} {count_B:>3} {bar_B:<5}")

# 3. 個人別の詳細
print("\n\n■ 参加者別の伸び（ソート済み）")
print("\n  A群:")
sorted_A = sorted(zip(gain_A, range(len(gain_A))), key=lambda x: x[0])
for g, i in sorted_A:
    bar = "+" * int(max(0, g/5)) if g >= 0 else "-" * int(abs(g/5))
    print(f"    参加者{i+1:2d}: {g:+6.1f}% {bar}")

print("\n  B群:")
sorted_B = sorted(zip(gain_B, range(len(gain_B))), key=lambda x: x[0])
for g, i in sorted_B:
    bar = "+" * int(max(0, g/5)) if g >= 0 else "-" * int(abs(g/5))
    print(f"    参加者{i+1:2d}: {g:+6.1f}% {bar}")

# 4. 伸びた人・伸びなかった人の分析
print("\n\n■ 伸びのカテゴリ別分析")
categories = [
    ("大幅低下", lambda x: x <= -10),
    ("やや低下", lambda x: (x > -10) & (x < 0)),
    ("変化なし", lambda x: x == 0),
    ("やや上昇", lambda x: (x > 0) & (x < 10)),
    ("大幅上昇", lambda x: x >= 10)
]

print("  " + "-" * 60)
print(f"  {'カテゴリ':<12} {'A群':>8} {'B群':>8} {'合計':>8}")
print("  " + "-" * 60)
for cat_name, condition in categories:
    count_A = np.sum(condition(gain_A))
    count_B = np.sum(condition(gain_B))
    print(f"  {cat_name:<12} {count_A:>8} {count_B:>8} {count_A + count_B:>8}")

# 5. Signal-to-Noise Ratio
print("\n\n■ シグナル対ノイズ比 (Signal-to-Noise Ratio)")
mean_diff = np.mean(gain_B) - np.mean(gain_A)  # 群間差（シグナル）
pooled_sd = np.sqrt((np.var(gain_A, ddof=1) + np.var(gain_B, ddof=1)) / 2)  # プールSD（ノイズ）
snr = abs(mean_diff) / pooled_sd

print(f"  群間差（シグナル）: {mean_diff:+.1f}%")
print(f"  群内SD（ノイズ）: {pooled_sd:.1f}%")
print(f"  SNR = |シグナル| / ノイズ = {snr:.3f}")
print(f"\n  解釈: SNRが低い（{snr:.3f} < 1）ため、")
print("        群間差が個人差のノイズに埋もれている")

In [ ]:
# 回帰分析
from scipy.stats import linregress

# 6. Pre得点と伸びの関係（詳細）
print("【Pre得点と伸びの関係（詳細分析）】")
print("=" * 70)

# 全体
slope, intercept, r_value, p_value, std_err = linregress(all_pre, all_gain)
print("\n■ 全体の回帰分析")
print(f"  回帰式: 伸び = {slope:.3f} × Pre + {intercept:.2f}")
print(f"  相関係数 r = {r_value:.3f}")
print(f"  決定係数 R² = {r_value**2:.3f}")
print(f"  p値 = {p_value:.4f}")

# 群別
print("\n■ 群別の回帰分析")
for group_name, pre, gain in [('A', pre_A, gain_A), ('B', pre_B, gain_B)]:
    slope_g, intercept_g, r_g, p_g, _ = linregress(pre, gain)
    print(f"\n  {group_name}群:")
    print(f"    回帰式: 伸び = {slope_g:.3f} × Pre + {intercept_g:.2f}")
    print(f"    r = {r_g:.3f}, R² = {r_g**2:.3f}, p = {p_g:.3f}")

# 予測される伸び（Pre得点別）
print("\n■ Pre得点別の予測される伸び（全体回帰式より）")
pre_levels = [50, 60, 70, 80, 90, 100]
print("  Pre得点\t予測伸び")
for pre_level in pre_levels:
    predicted_gain = slope * pre_level + intercept
    print(f"  {pre_level}%\t\t{predicted_gain:+.1f}%")

# 天井効果の影響の定量化
print("\n■ 天井効果の影響")
# Preが高い人（85%以上）を除いた場合の群間比較
high_threshold = 85
mask_A = pre_A < high_threshold
mask_B = pre_B < high_threshold

print(f"  高得点者 (Pre≥{high_threshold}%) を除外:")
print(f"    A群: n={np.sum(mask_A)}, 伸び = {np.mean(gain_A[mask_A]):+.1f}% (SD={np.std(gain_A[mask_A], ddof=1):.1f}%)")
print(f"    B群: n={np.sum(mask_B)}, 伸び = {np.mean(gain_B[mask_B]):+.1f}% (SD={np.std(gain_B[mask_B], ddof=1):.1f}%)")

if np.sum(mask_A) > 1 and np.sum(mask_B) > 1:
    t_excl, p_excl = stats.ttest_ind(gain_A[mask_A], gain_B[mask_B], alternative="less")
    d_excl = (np.mean(gain_A[mask_A]) - np.mean(gain_B[mask_B])) / np.sqrt(
        ((np.sum(mask_A)-1)*np.var(gain_A[mask_A], ddof=1) + 
         (np.sum(mask_B)-1)*np.var(gain_B[mask_B], ddof=1)) / 
        (np.sum(mask_A) + np.sum(mask_B) - 2))
    print(f"    t検定: t = {t_excl:.3f}, p = {p_excl:.3f}")
    print(f"    Cohen's d = {d_excl:.3f}")

# 低得点者のみの分析
low_threshold = 70
mask_A_low = pre_A <= low_threshold
mask_B_low = pre_B <= low_threshold

print(f"\n  低得点者 (Pre≤{low_threshold}%) のみ:")
print(f"    A群: n={np.sum(mask_A_low)}, 伸び = {np.mean(gain_A[mask_A_low]):+.1f}% (SD={np.std(gain_A[mask_A_low], ddof=1):.1f}%)")
print(f"    B群: n={np.sum(mask_B_low)}, 伸び = {np.mean(gain_B[mask_B_low]):+.1f}% (SD={np.std(gain_B[mask_B_low], ddof=1):.1f}%)")

if np.sum(mask_A_low) > 1 and np.sum(mask_B_low) > 1:
    t_low, p_low = stats.ttest_ind(gain_A[mask_A_low], gain_B[mask_B_low], alternative="less")
    print(f"    t検定: t = {t_low:.3f}, p = {p_low:.3f}")

### 14. 有意差が見られなかった原因の総合まとめ

In [ ]:
# 有意差が見られなかった原因の総合まとめ
print("=" * 70)
print("【有意差が見られなかった原因の総合まとめ】")
print("=" * 70)

# 計算値の取得
power_current = power_t_test_ind(10, 10, abs(compute_cohens_d(gain_A, gain_B)))
snr = abs(np.mean(gain_B) - np.mean(gain_A)) / np.sqrt((np.var(gain_A, ddof=1) + np.var(gain_B, ddof=1)) / 2)
corr, _ = stats.pearsonr(all_pre, all_gain)

print("""
┌────────────────────────────────────────────────────────────────────────┐
│                      有意差が見られなかった原因                         │
├────────────────────────────────────────────────────────────────────────┤
│                                                                        │
│  【原因1】検出力不足（最も重要）                                        │
│  ─────────────────────────────────────────────────────────────────────│""")
print(f"│    ・現在の検出力: {power_current*100:.1f}% （推奨: 80%以上）                          │")
print(f"│    ・観測された効果量 d={abs(compute_cohens_d(gain_A, gain_B)):.3f} を検出するには各群n={required_sample_size(abs(compute_cohens_d(gain_A, gain_B)))}必要   │")
print("""│    ・現在のn=10では大きな効果(d≥0.8)しか検出できない                 │
│                                                                        │
│  【原因2】個人差が大きい                                                │
│  ─────────────────────────────────────────────────────────────────────│""")
print(f"│    ・群内標準偏差: A群={np.std(gain_A, ddof=1):.1f}%, B群={np.std(gain_B, ddof=1):.1f}%                        │")
print(f"│    ・群間差（シグナル）: {np.mean(gain_B) - np.mean(gain_A):+.1f}%                                   │")
print(f"│    ・SNR = {snr:.3f} （シグナルがノイズに埋もれている）                   │")
print("""│                                                                        │
│  【原因3】天井効果                                                      │
│  ─────────────────────────────────────────────────────────────────────│""")
print(f"│    ・Pre得点と伸びの相関: r = {corr:.3f} （有意な負の相関）               │")
print("""│    ・高得点者（Pre≥85%）は伸びにくい傾向                              │
│    ・Pre平均が両群とも70%以上と高く、伸びる余地が限定的                 │
│                                                                        │
│  【原因4】そもそも効果が小さい可能性                                    │
│  ─────────────────────────────────────────────────────────────────────│
│    ・ベイズ分析では「差がない」を支持 (BF01 > 1)                        │
│    ・AとBの学習法に本質的な差がない可能性も考慮すべき                   │
│                                                                        │
└────────────────────────────────────────────────────────────────────────┘
""")

print("【推奨される対策】")
print("=" * 70)
print("""
1. サンプルサイズの増加
   → 中程度の効果(d=0.5)を検出するには各群n=64が必要
   → 観測された効果(d=0.196)を検出するには各群n=410が必要

2. 参加者の選定基準の見直し
   → Pre得点が低い参加者（60-70%）を対象にすることで天井効果を回避
   → 伸びる余地のある参加者で効果を検証

3. 測定精度の向上
   → テスト問題数を増やす（現在20問→40問など）
   → 個人差のノイズを減らす

4. 効果の再検討
   → ベイズ分析の結果も踏まえ、本当に効果があるのか理論的に再検討
   → 他の指標（解答時間、視線データ）での効果も確認
""")

## スロープグラフ（Pre → Post 正答数の変化）

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['font.family'] = 'Hiragino Sans'

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 6), sharey=True)

group_colors = {"A": "#4C72B0", "B": "#DD8452"}
titles = {"A": "比較群", "B": "実験群"}
axes = {"A": ax2, "B": ax1}

for group in ["A", "B"]:
    ax = axes[group]
    gc = group_colors[group]
    participants = sorted(
        p for p in data[group]
        if "pre" in data[group][p] and "post" in data[group][p]
    )
    pre_rates = []
    post_rates = []

    for participant in participants:
        p_data = data[group][participant]
        pre_r = p_data["pre"]["rate"]
        post_r = p_data["post"]["rate"]
        pre_rates.append(pre_r)
        post_rates.append(post_r)
        ax.plot(
            [0, 1], [pre_r, post_r],
            color=gc, alpha=0.3, linewidth=1.2,
            marker="o", markersize=4, zorder=2,
        )

    # 群平均線
    mean_pre = np.mean(pre_rates)
    mean_post = np.mean(post_rates)
    ax.plot(
        [0, 1], [mean_pre, mean_post],
        color=gc, alpha=1.0, linewidth=3.0,
        marker="o", markersize=10, zorder=3,
        label="平均",
    )
    ax.annotate(
        f"{mean_pre:.1f}%", (0, mean_pre),
        textcoords="offset points", xytext=(-38, 0),
        fontsize=10, color=gc, fontweight="bold", va="center",
    )
    ax.annotate(
        f"{mean_post:.1f}%", (1, mean_post),
        textcoords="offset points", xytext=(12, 0),
        fontsize=10, color=gc, fontweight="bold", va="center",
    )

    # 中央の縦点線
    ax.axvline(x=0.5, color="gray", linestyle=":", linewidth=0.8, zorder=1)

    ax.set_title(titles[group], fontsize=14, fontweight="bold")
    ax.set_xticks([0, 1])
    ax.set_xticklabels(["Pre", "Post"], fontsize=12)
    ax.set_xlim(-0.3, 1.3)
    ax.set_ylim(0, 100)

    # 枠線: 黒で囲む
    for spine in ax.spines.values():
        spine.set_visible(True)
        spine.set_color("black")
        spine.set_linewidth(0.8)
    ax.tick_params(bottom=False)

    # Y軸グリッド線
    ax.yaxis.grid(True, alpha=0.3, linestyle="--")
    ax.set_axisbelow(True)

    ax.legend(loc="lower right", fontsize=10)

ax1.set_ylabel("正答率 (%)", fontsize=12)

fig.suptitle("Pre/Post テスト正答率の変化", fontsize=15, fontweight="bold", y=0.98)
plt.tight_layout(rect=[0, 0, 1, 0.94])

output_path = project_root / "data" / "output" / "slopegraph_pre_post.png"
output_path.parent.mkdir(parents=True, exist_ok=True)
fig.savefig(output_path, dpi=300, bbox_inches="tight", facecolor="white")
print(f"保存完了: {output_path}")
plt.show()